In [12]:
import numpy as np

In [13]:
P = 9                               # # of channels
T = 1                               # serve time (1 * 1)
L = np.arange(P * P).reshape(P, P)  # lambda_ij (P * P)
LC = np.diag(np.arange(P))          # lamdba_i (P * P)
R = [(T * T * LC[i, i] / 2) for i in range(P)]

In [14]:
diag_idx = [i for i in range(P)]
F = L / L.sum(axis=0).reshape(P, 1)
F[diag_idx, diag_idx] = 0

C = np.dot(F, F.T)
C[diag_idx, diag_idx] = 1

In [16]:
I = np.eye(P)
N = I - np.dot(T * LC, C)
N = np.linalg.inv(N)
N = np.dot(np.dot(N, LC), R)
LC_1D = np.diag(LC)
N / LC_1D

array([        nan,  5.60566781, -0.31693828, -0.29462546, -0.29999527,
       -0.31419872, -0.33387556, -0.35833866, -0.38780642])

In [62]:
# Routing Algorithm Simulation: XY
import numpy as np

channel_size = 4
router_size = 4
P = channel_size
N = np.sqrt(router_size)
task_graph = [(0, 3, 10), (2, 1, 3)]    # src, dst, volume
LC = np.zeros(channel_size)
L = np.zeros((channel_size, channel_size))

In [64]:
index2cord = lambda num: (int(num // N), int(num % N))
f2 = lambda l, r: (min(l, r), max(l, r))
cord2index = lambda x, y: x * N + y

path = [0, 1, 3]
L[path[:-1], path[1:]] += 1
print(L)
# for src, dst, vol in task_graph:
#     src_cord_x, src_cord_y = f1(src)
#     dst_cord_x, dst_cord_y = f1(dst)
#     start_x, last_x = f2(src_cord_x, dst_cord_x)
#     start_y, last_y = f2(src_cord_y, dst_cord_y)
#     print("y:", start_y, last_y)
#     print("x:", start_x, last_x)
#     path_index = [f3(src_cord_x, y) for y in range(start_y, last_y + 1)]
#     path_index += [f3(x, dst_cord_y) for x in range(start_x, last_x + 1)]
#     path_index = list(set(path_index))  # 去重
#     print(path_index)

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [66]:
task_graph = [(0, 3, 10), (2, 1, 3)]


n = 2

def index2cord(num):
    return (int(num // n), int(num % n))

def routerCord2channelIndex(x, y, direction):
    bias = {"east": 0, "west": -1, "north": -n, "south": n - 1}
    return x * (2 * n - 1) + y + bias.get(direction)

cord_task_graph = [
    (index2cord(item[0]), index2cord(item[1]), item[2])
    for item in task_graph
]

cs = 4
L = np.zeros((cs, cs))
LC = np.zeros(cs)
for request in cord_task_graph:
    src_x, src_y = request[0]
    dst_x, dst_y = request[1]
    V = request[2]
    # x routing
    step = 1 if src_y < dst_y else -1
    direction = "west" if src_y < dst_y else "east"
    path = [
        routerCord2channelIndex(src_x, y, direction)
        for y in range(src_y, dst_y, step)
    ]
    # y routing
    step = 1 if src_x < dst_x else -1
    direction = "south" if src_x < dst_x else "north"
    path += [
        routerCord2channelIndex(x, dst_y, direction)
        for x in range(src_x, dst_x, step)
    ]
    # update
    L[path[:-1], path[1:]] += V
    LC[path] += V

    print(path)

diag_index = [i for i in range(cs)]
L[diag_index, diag_index] = 0


[-1, 2]
[2, 2]
